In [1]:
from utils import mnist_reader
from sklearn import svm
import numpy as np
from sklearn.metrics import accuracy_score
import cvxpy as cp 
import cv2
from matplotlib import pyplot as plt

In [2]:
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [3]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [4]:
def training(X_train, y_train, X_test, y_test):
    # Train SVM classifier with default hyperparameters
    print("Train On Dataset")
    clf = svm.SVC()
    clf.fit(X_train, y_train)

    # Evaluate the classifier on the test set
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print('Test accuracy: ', acc)

In [5]:
# front = 0
# last = 1
# X_train = X_train[front:last]
# X_test = X_test[front:last]
# y_train = y_train[front:last]
# y_test = y_test[front:last]

i = 0
x_train_opt = []
x_test_opt = []
for i in range(len(X_train)):
	cv2.imwrite(f'data/train_test/train_img_{i}.png', X_train[i])
	img = cv2.imread(f'data/train_test/train_img_{i}.png')
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

	smoothed_X = cp.Variable(gray.shape)
	beta = 0.002

	# Define the objective function
	objective = cp.norm(smoothed_X - gray, 'fro') ** 2 + beta * cp.norm(np.cov(gray), 1)
	constraint = [0 <= smoothed_X, smoothed_X <= 255]

	# Define the problem and solve it
	problem = cp.Problem(cp.Minimize(objective), constraint)
	problem.solve()

	smoothed_X = np.array(smoothed_X.value * 255.0, dtype = np.uint8)

	th2 = cv2.adaptiveThreshold(smoothed_X, 255,
		cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 57, 10)

	x_train_opt.append(th2)

for i in range(len(X_test)):
	cv2.imwrite(f'data/train_test/test_img_{i}.png', X_test[i])
	img = cv2.imread(f'data/train_test/test_img_{i}.png')
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

	smoothed_X = cp.Variable(gray.shape)
	beta = 0.002

	# Define the objective function
	objective = cp.norm(smoothed_X - gray, 'fro') ** 2 + beta * cp.norm(np.cov(gray), 1)
	constraint = [0<=smoothed_X, smoothed_X<=255]
	
	# Define the problem and solve it
	problem = cp.Problem(cp.Minimize(objective), constraint)
	problem.solve()

	smoothed_X = np.array(smoothed_X.value * 255.0, dtype = np.uint8)

	th2 = cv2.adaptiveThreshold(smoothed_X, 255,
		cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 57, 10)

	x_test_opt.append(th2)

x_train_opt = np.array(x_train_opt).reshape(-1, 784)
x_test_opt = np.array(x_test_opt).reshape(-1, 784)
print(x_train_opt.shape)
training(x_train_opt, y_train, x_test_opt, y_test)

/Users/jimwan/Library/Python/3.10/lib/python/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [ ]:
X_train = X_train.reshape((-1, 784))
X_test = X_test.reshape((-1, 784))
training(X_train, y_train, X_test, y_test)

Train On Dataset
Test accuracy:  0.8828
